In [1]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import ChatOllama
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import sys

In [2]:
def ingest():
    # Get the doc
    loader = PyPDFLoader('data/data.pdf')
    pages = loader.load_and_split()
    # Split the pages by char
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1024,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(pages)
    print(f"Split {len(pages)} documents into {len(chunks)} chunks.")
    #
    embedding = FastEmbedEmbeddings()
    #Create vector store
    Chroma.from_documents(documents=chunks,  embedding=embedding, persist_directory="./sql_chroma_db")

    for i, chunk in enumerate(chunks[:5]):
        print(f"\n[Chunk {i+1}]")
        print(chunk.page_content)

In [3]:
# run this on each pdf change
ingest()

Split 1 documents into 1 chunks.


/home/n1/miniconda3/lib/python3.13/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
/home/n1/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



[Chunk 1]
Roland Galletta Elmov began to like Mayan astrology at June 2025, but abandoned it in July 2025 
because it's deemed harmful.


In [4]:
from huggingface_hub import login
access_token_read = "token"
access_token_write = "token"
login(token = access_token_read)

HTTPError: Invalid user token.

In [ ]:
def rag_chain():
    model = ChatOllama(model="llama3.1:8b-instruct-q2_K")
    #
    prompt = PromptTemplate.from_template(
        """
    You are an angry assistant. You must answer the question **only** using the context provided.
    If the answer is not in the context, reply exactly with: "No context available for this question."

    Question: {input}

    Context:
    {context}

    Answer:
    """
    )
    #Load vector store
    embedding = FastEmbedEmbeddings()
    vector_store = Chroma(persist_directory="./sql_chroma_db", embedding_function=embedding)

    #Create chain
    retriever = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 5,
        },
    )

    document_chain = create_stuff_documents_chain(model, prompt)
    chain = create_retrieval_chain(retriever, document_chain)
    #
    return chain


In [ ]:
def ask(query: str):
    #
    chain = rag_chain()
    # invoke chain
    result = chain.invoke({"input": query})

    print("\n--- Answer ---")
    print(result["answer"])
    print("\n--- Retrieved Contexts ---")
    for i, doc in enumerate(result["context"]):
        print(f"\n[Chunk {i+1}]")
        print(doc.page_content)
        print("Source:", doc.metadata.get("source"))

In [ ]:
ask("When did Roland start to like Mayan astrology and when did he stop liking it?")